In [ ]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import re
from typing import List, Union, Dict
import warnings
warnings.filterwarnings('ignore')

# Определение каналов ЭЭГ
EXPECTED_CHANNELS = [
    'A1', 'A2',             # Референтные электроды
    'C3', 'C4', 'Cz',      # Центральные
    'F3', 'F4', 'F7', 'F8', # Фронтальные
    'Fp1', 'Fp2', 'Fpz',    # Префронтальные
    'Fz',                   # Фронтальный средний
    'O1', 'O2',            # Затылочные
    'P3', 'P4', 'Pz',      # Теменные
    'T3', 'T4', 'T5', 'T6' # Височные
]

def parse_array_string(s: str) -> np.ndarray:
    """Преобразование строки с массивом в numpy array."""
    try:
        # Очистка строки
        s = s.strip()
        if s.startswith('[') and s.endswith(']'):
            # Удаляем скобки и разделяем по запятой
            values = s[1:-1].split(',')
            # Конвертируем в float
            return np.array([float(v.strip()) for v in values], dtype=np.float32)
        return np.array([float(s)], dtype=np.float32)
    except Exception as e:
        print(f"Error parsing: {s[:50]}... Error: {str(e)}")
        return np.array([np.nan], dtype=np.float32)

def process_channel_value(value: Union[str, float, int]) -> np.ndarray:
    """
    Обработка значения канала ЭЭГ.
    Возвращает временной ряд как numpy array.
    """
    if pd.isna(value):
        return np.array([np.nan])
    
    try:
        if isinstance(value, (int, float)):
            return np.array([float(value)])
        
        if isinstance(value, str):
            return parse_array_string(value)
        
        return np.array([np.nan])
    
    except Exception as e:
        print(f"Error processing value: {str(e)}")
        return np.array([np.nan])

def validate_data(data: np.ndarray, name: str = "") -> np.ndarray:
    """Валидация и очистка данных."""
    print(f"\nValidating {name}:")
    print(f"Shape: {data.shape}")
    print(f"dtype: {data.dtype}")
    
    # Проверка на некорректные значения
    nan_mask = np.isnan(data)
    inf_mask = np.isinf(data)
    
    if nan_mask.any() or inf_mask.any():
        print(f"Found invalid values:")
        print(f"NaN count: {nan_mask.sum()}")
        print(f"Inf count: {inf_mask.sum()}")
        
        # Заменяем некорректные значения
        invalid_mask = nan_mask | inf_mask
        valid_data = data[~invalid_mask]
        
        if valid_data.size > 0:
            fill_value = np.mean(valid_data)
        else:
            fill_value = 0.0
            
        data[invalid_mask] = fill_value
        
    return data.astype(np.float32)


In [ ]:
# Загрузка и подготовка данных
print("Loading data...")
try:
    # Загрузка данных
    df = pd.read_csv('/home/belyaeva.a/df_open.csv')
    print("DataFrame shape:", df.shape)
    
    # Создание словаря для преобразования меток в индексы
    unique_labels = df['label'].unique()
    label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}
    print("\nFound classes:", unique_labels.tolist())
    
    # Предварительная обработка данных
    print("\nPreprocessing data...")
    processed_channels = {}
    sequence_lengths = []  # Для отслеживания длины временных рядов
    
    # Обработка каждого канала отдельно
    for col in EXPECTED_CHANNELS:
        print(f"\nProcessing channel {col}...")
        channel_data = df[col].copy()
        
        # Анализ данных канала
        print(f"Channel type: {channel_data.dtype}")
        print(f"NaN values: {channel_data.isna().sum()}")
        
        # Преобразование данных канала
        channel_sequences = []
        for value in channel_data:
            seq = process_channel_value(value)
            channel_sequences.append(seq)
            sequence_lengths.append(len(seq))
        
        processed_channels[col] = channel_sequences
    
    # Определение максимальной длины последовательности
    max_seq_length = max(sequence_lengths)
    min_seq_length = min(sequence_lengths)
    print(f"\nSequence length statistics:")
    print(f"Min length: {min_seq_length}")
    print(f"Max length: {max_seq_length}")
    
    # Создание трехмерного массива (n_samples, n_channels, sequence_length)
    n_samples = len(df)
    n_channels = len(EXPECTED_CHANNELS)
    
    # Инициализация массива данных
    X = np.zeros((n_samples, n_channels, max_seq_length), dtype=np.float32)
    
    # Заполнение массива данными
    for i, channel in enumerate(EXPECTED_CHANNELS):
        for j, seq in enumerate(processed_channels[channel]):
            # Если последовательность короче максимальной, заполняем оставшееся место последним значением
            seq_len = len(seq)
            X[j, i, :seq_len] = seq
            if seq_len < max_seq_length:
                X[j, i, seq_len:] = seq[-1]
    
    # Преобразование меток
    y = np.array([label_to_idx[label] for label in df['label']])
    
    print("\nFinal data information:")
    print(f"X shape: {X.shape}")  # (n_samples, n_channels, sequence_length)
    print(f"X dtype: {X.dtype}")
    print(f"Sample of X (first channel, first 5 timepoints):\n{X[0, 0, :5]}")
    print(f"Label mapping:", label_to_idx)
    print(f"y shape: {y.shape}")
    
    # Сохранение обработанных данных
    np.save('processed_X.npy', X)
    np.save('processed_y.npy', y)
    np.save('label_mapping.npy', label_to_idx)
    print("\nProcessed data saved to 'processed_X.npy', 'processed_y.npy', and 'label_mapping.npy'")
    
except Exception as e:
    print(f"\nError during data loading and preprocessing: {str(e)}")
    print("\nDetailed error information:")
    if 'df' in locals():
        print(f"DataFrame shape: {df.shape}")
        print(f"Column types:\n{df[EXPECTED_CHANNELS].dtypes}")
        print("\nSample of raw data:")
        for col in EXPECTED_CHANNELS[:2]:
            print(f"\n{col}:")
            print(df[col].iloc[0])
    raise
